In [57]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
plt.style.use('seaborn')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.min_rows', 500)
import warnings
warnings.filterwarnings('ignore')

path='../../data/'
train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")
sub = pd.read_csv(f"{path}/sample_submission.csv")

In [58]:
train_test = pd.concat([train, test])

In [61]:
train_test.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')

In [78]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
plt.style.use('seaborn')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.min_rows', 500)
import warnings
warnings.filterwarnings('ignore')

def concat_and_export(train_fe, test_fe):
    
    train_fe['kind']='train'
    test_fe['kind']='test'
    
    df = pd.concat([train_fe,test_fe])
    data_dir = '/opt/ml/input/data' # 경로
    write_path=f'{data_dir}/after_fe_train_test.pkl'
    df.to_pickle(write_path)
    print(f"Write: {write_path}")
    
def export(df):
    data_dir = '/opt/ml/input/data' # 경로
    write_path=f'{data_dir}/after_fe_train_test.pkl'
    df.to_pickle(write_path)
    print(f"Write: {write_path}")
    
path='../../data/'
train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")
sub = pd.read_csv(f"{path}/sample_submission.csv")
test = test[test['answerCode'] != -1]

day_dict = {'Tuesday': 0,
 'Thursday': 1,
 'Monday': 2,
 'Saturday': 3,
 'Friday': 4,
 'Wednesday': 5,
 'Sunday': 6}

def feature_engineering(df):
    df2 = df.copy()
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df2.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df2['user_correct_answer'] = df2.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df2['user_total_answer'] = df2.groupby('userID')['answerCode'].cumcount()
    df2['user_acc'] = df2['user_correct_answer']/df2['user_total_answer']
    df2['month'] = pd.to_datetime(df2.Timestamp).dt.month
    df2['day'] = pd.to_datetime(df2.Timestamp).dt.day
    df2['hour'] = pd.to_datetime(df2.Timestamp).dt.hour
    df2['dayname'] = pd.to_datetime(df2.Timestamp).dt.day_name().map(day_dict)
    df2['big_category'] = df2.testId.map(lambda x:x[2]).astype(int)
    df2['problem_num'] = df2.assessmentItemID.map(lambda x: int(x[-3:]))
    df2['mid_category'] = df2.testId.map(lambda x: int(x[-3:]))

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df2.groupby(['testId'])['answerCode'].agg(['mean'])
    correct_t.columns = ["test_mean"]
    correct_k = df2.groupby(['KnowledgeTag'])['answerCode'].agg(['mean'])
    correct_k.columns = ["tag_mean"]
    correct_q = df2.groupby(['assessmentItemID'])['answerCode'].agg(['mean'])
    correct_q.columns = ["assess_mean"]

    df2 = pd.merge(df2, correct_t, on=['testId'], how="left")
    df2 = pd.merge(df2, correct_k, on=['KnowledgeTag'], how="left")
    df2 = pd.merge(df2, correct_q, on=['assessmentItemID'], how="left")
    
    return df2

train_fe = feature_engineering(train)
test_fe = feature_engineering(test)
train_fe.shape, test_fe.shape

train_fe['kind']='train'
test_fe['kind']='test'
columns = ['userID','answerCode','KnowledgeTag','user_acc','month','hour','big_category','mid_category','problem_num','assess_mean']
df = pd.concat([train_fe,test_fe])[columns]

((2266586, 19), (259370, 19))

In [79]:
df['user_acc'] = df['user_acc'].fillna((df['assess_mean']))

In [80]:
df

,userID,answerCode,KnowledgeTag,user_acc,month,hour,big_category,mid_category,problem_num,assess_mean
0,0,1,7224,0.982063,3,0,6,1,1,0.982063
1,0,1,7225,1.000000,3,0,6,1,2,0.964126
2,0,1,7225,1.000000,3,0,6,1,3,0.910314
3,0,1,7225,1.000000,3,0,6,1,4,0.968610
4,0,1,7225,1.000000,3,0,6,1,5,0.941704
5,0,1,7225,1.000000,3,0,6,1,7,0.919283
6,0,0,7226,1.000000,3,5,6,3,1,0.882353
7,0,1,7226,0.857143,3,5,6,3,2,0.918552
8,0,1,7226,0.875000,3,5,6,3,3,0.520362
9,0,1,7226,0.888889,3,5,6,3,4,0.823529


In [ ]:
def feature_engineering(df):
    df.sort_values(['userID','Timestamp'])
    df.reset_index(drop=True, inplace=True)
    df['Timestamp_start'] = pd.to_datetime(df['Timestamp'])
    df['Timestamp_fin'] = df.groupby('userID')['Timestamp_start'].shift(-1)
    df['solvetime'] = df.Timestamp_fin - df.Timestamp_start
    df['solvesec'] = df.solvetime.map(lambda x : x.total_seconds())
    ques_sec_mean = df.groupby(['assessmentItemID'])['solvesec'].agg(['mean'])
    ques_sec_mean.columns = ["ques_sec_mean"]
    df = pd.merge(df, ques_sec_mean, on=['assessmentItemID'], how="left")
    df['ques_sec_mean'] = df['ques_sec_mean']
    df.solvesec = df.solvesec.fillna(df['ques_sec_mean'])
    df.loc[df.solvesec>=3600,'solvesec']=3600
    return df[['userID','assessmentItemID','testId','answerCode','KnowledgeTag','solvesec']]

In [94]:
train_ori = pd.read_csv(f"{path}/train_data.csv")

In [180]:
pd.set_option('display.min_rows',10)

In [161]:
train = train_ori.copy()

In [162]:
train = train.sort_values(['userID','Timestamp'])
train.reset_index(drop=True, inplace=True)

In [163]:
train['Timestamp_start'] = pd.to_datetime(train['Timestamp'])
train['Timestamp_fin'] = train.groupby('userID')['Timestamp_start'].shift(-1)
train['solvetime'] = train.Timestamp_fin - train.Timestamp_start
train['solvesec'] = train.solvetime.map(lambda x : x.total_seconds())
ques_sec_mean = train.groupby(['assessmentItemID'])['solvesec'].agg(['mean'])
ques_sec_mean.columns = ["ques_sec_mean"]
train = pd.merge(train, ques_time_mean, on=['assessmentItemID'], how="left")

In [164]:
train['ques_time_mean'] = train['ques_time_mean'].dt.total_seconds()

In [165]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2266586 entries, 0 to 2266585
Data columns (total 11 columns):
 #   Column            Dtype          
---  ------            -----          
 0   userID            int64          
 1   assessmentItemID  object         
 2   testId            object         
 3   answerCode        int64          
 4   Timestamp         object         
 5   KnowledgeTag      int64          
 6   Timestamp_start   datetime64[ns] 
 7   Timestamp_fin     datetime64[ns] 
 8   solvetime         timedelta64[ns]
 9   solvesec          float64        
 10  ques_time_mean    float64        
dtypes: datetime64[ns](2), float64(2), int64(3), object(3), timedelta64[ns](1)
memory usage: 207.5+ MB


In [166]:
train2 = train.copy()

,userID,assessmentItemID,testId,answerCode,Timestamp,...,Timestamp_start,Timestamp_fin,solvetime,solvesec,ques_time_mean
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,...,2020-03-24 00:17:11,2020-03-24 00:17:14,0 days 00:00:03,3.0,13.762332
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,...,2020-03-24 00:17:14,2020-03-24 00:17:22,0 days 00:00:08,8.0,26.708520
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,...,2020-03-24 00:17:22,2020-03-24 00:17:29,0 days 00:00:07,7.0,19.237668
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,...,2020-03-24 00:17:29,2020-03-24 00:17:36,0 days 00:00:07,7.0,18.811659
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,...,2020-03-24 00:17:36,2020-03-24 00:17:47,0 days 00:00:11,11.0,34.529148
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,...,2020-03-24 00:17:47,2020-03-26 05:52:03,2 days 05:34:16,192856.0,426199.811659
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,...,2020-03-26 05:52:03,2020-03-26 05:52:10,0 days 00:00:07,7.0,41.407240
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,...,2020-03-26 05:52:10,2020-03-26 05:53:14,0 days 00:01:04,64.0,107.606335
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,...,2020-03-26 05:53:14,2020-03-26 05:53:29,0 days 00:00:15,15.0,2024.171946
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,...,2020-03-26 05:53:29,2020-03-26 05:53:48,0 days 00:00:19,19.0,136.208145


In [171]:
train.solvesec = train.solvesec.fillna(train['ques_time_mean'])
train.loc[train.solvesec>=3600,'solvesec']=3600

In [172]:

train

,userID,assessmentItemID,testId,answerCode,Timestamp,...,Timestamp_start,Timestamp_fin,solvetime,solvesec,ques_time_mean
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,...,2020-03-24 00:17:11,2020-03-24 00:17:14,0 days 00:00:03,3.0,13.762332
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,...,2020-03-24 00:17:14,2020-03-24 00:17:22,0 days 00:00:08,8.0,26.708520
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,...,2020-03-24 00:17:22,2020-03-24 00:17:29,0 days 00:00:07,7.0,19.237668
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,...,2020-03-24 00:17:29,2020-03-24 00:17:36,0 days 00:00:07,7.0,18.811659
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,...,2020-03-24 00:17:36,2020-03-24 00:17:47,0 days 00:00:11,11.0,34.529148
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,...,2020-03-24 00:17:47,2020-03-26 05:52:03,2 days 05:34:16,3600.0,426199.811659
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,...,2020-03-26 05:52:03,2020-03-26 05:52:10,0 days 00:00:07,7.0,41.407240
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,...,2020-03-26 05:52:10,2020-03-26 05:53:14,0 days 00:01:04,64.0,107.606335
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,...,2020-03-26 05:53:14,2020-03-26 05:53:29,0 days 00:00:15,15.0,2024.171946
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,...,2020-03-26 05:53:29,2020-03-26 05:53:48,0 days 00:00:19,19.0,136.208145


In [ ]:
#월별로 개수 히스토그램 그래프로 나타내기
train_df['Timestamp'] = train_df['Timestamp'].astype("datetime64")
train_df.groupby(train_df['Timestamp'].dt.month)['answerCode'].count().plot(kind="bar")

In [168]:
train2.drop(['Timestamp_start','Timestamp_fin','solvetime','ques_time_mean'], axis=1)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solvesec
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,3.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,8.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,7.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,11.0
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,192856.0
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,7.0
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,64.0
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,15.0
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,19.0


In [144]:
train.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'Timestamp_start', 'Timestamp_fin', 'solvetime',
       'solvesec', 'ques_time_mean'],
      dtype='object')

In [186]:
df = train_ori.copy()

In [187]:
df.sort_values(['userID','Timestamp'])
df.reset_index(drop=True, inplace=True)
df['Timestamp_start'] = pd.to_datetime(df['Timestamp'])
df['Timestamp_fin'] = df.groupby('userID')['Timestamp_start'].shift(-1)
df['solvetime'] = df.Timestamp_fin - df.Timestamp_start
df['solvesec'] = df.solvetime.map(lambda x : x.total_seconds())
ques_sec_mean = df.groupby(['assessmentItemID'])['solvesec'].agg(['mean'])
ques_sec_mean.columns = ["ques_sec_mean"]
df = pd.merge(df, ques_sec_mean, on=['assessmentItemID'], how="left")
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


,userID,assessmentItemID,testId,answerCode,Timestamp,...,Timestamp_start,Timestamp_fin,solvetime,solvesec,ques_sec_mean
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,...,2020-03-24 00:17:11,2020-03-24 00:17:14,0 days 00:00:03,3.0,13.762332
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,...,2020-03-24 00:17:14,2020-03-24 00:17:22,0 days 00:00:08,8.0,26.708520
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,...,2020-03-24 00:17:22,2020-03-24 00:17:29,0 days 00:00:07,7.0,19.237668
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,...,2020-03-24 00:17:29,2020-03-24 00:17:36,0 days 00:00:07,7.0,18.811659
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,...,2020-03-24 00:17:36,2020-03-24 00:17:47,0 days 00:00:11,11.0,34.529148
...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,...,2020-06-05 06:50:21,2020-08-21 01:06:39,76 days 18:16:18,6632178.0,530916.830325
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,...,2020-08-21 01:06:39,2020-08-21 01:06:50,0 days 00:00:11,11.0,2475.985401
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,...,2020-08-21 01:06:50,2020-08-21 01:07:36,0 days 00:00:46,46.0,561.989051
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,...,2020-08-21 01:07:36,2020-08-21 01:08:49,0 days 00:01:13,73.0,1503.678832


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2266586 entries, 0 to 2266585
Data columns (total 12 columns):
 #   Column            Dtype          
---  ------            -----          
 0   userID            int64          
 1   assessmentItemID  object         
 2   testId            object         
 3   answerCode        int64          
 4   Timestamp         object         
 5   KnowledgeTag      int64          
 6   Timestamp_start   datetime64[ns] 
 7   Timestamp_fin     datetime64[ns] 
 8   solvetime         timedelta64[ns]
 9   solvesec          float64        
 10  ques_sec_mean_x   float64        
 11  ques_sec_mean_y   float64        
dtypes: datetime64[ns](2), float64(3), int64(3), object(3), timedelta64[ns](1)
memory usage: 224.8+ MB


In [193]:
df['ques_sec_mean'] = df['ques_sec_mean']
df.solvesec = df.solvesec.fillna(df['ques_sec_mean'])
df.loc[df.solvesec>=3600,'solvesec']=3600
df[['userID','assessmentItemID','testId','answerCode','KnowledgeTag','solvesec']]

,userID,assessmentItemID,testId,answerCode,KnowledgeTag,solvesec
0,0,A060001001,A060000001,1,7224,3.0
1,0,A060001002,A060000001,1,7225,8.0
2,0,A060001003,A060000001,1,7225,7.0
3,0,A060001004,A060000001,1,7225,7.0
4,0,A060001005,A060000001,1,7225,11.0
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,438,3600.0
2266582,7441,A040165001,A040000165,1,8836,11.0
2266583,7441,A040165002,A040000165,1,8836,46.0
2266584,7441,A040165003,A040000165,1,8836,73.0


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2266586 entries, 0 to 2266585
Data columns (total 11 columns):
 #   Column            Dtype          
---  ------            -----          
 0   userID            int64          
 1   assessmentItemID  object         
 2   testId            object         
 3   answerCode        int64          
 4   Timestamp         object         
 5   KnowledgeTag      int64          
 6   Timestamp_start   datetime64[ns] 
 7   Timestamp_fin     datetime64[ns] 
 8   solvetime         timedelta64[ns]
 9   solvesec          float64        
 10  ques_sec_mean     float64        
dtypes: datetime64[ns](2), float64(2), int64(3), object(3), timedelta64[ns](1)
memory usage: 207.5+ MB


In [176]:
df['ques_sec_mean'] = df['ques_sec_mean'].dt.total_seconds()
df.solvesec = df.solvesec.fillna(df['ques_sec_mean'])
df.loc[df.solvesec>=3600,'solvesec']=3600
df['userID','assessmentItemID','testId','answerCode','KnowledgeTag','solvesec']

KeyError: 'ques_sec_mean'